In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import catboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [56]:
df = pd.read_csv('./data/DatasetV4+.csv')
df.head()

,Client ID,Gender,Year,Trip Type,Order Type,Country,Region,Star Rate,Group Type,Meal Option,...,lat_from,lng_from,lat_to,lng_to,Birthday,Order,Order Date,Trip Start Date,Price,Booking Depth
0,4197429,1,3,0,0,0,0,2,0,1,...,59.939043,30.315816,36.719216,31.568803,672537600.0,4803671,1.612731e+09,1.620086e+09,1718.64,85.136111
1,4197517,1,3,0,0,0,5,1,0,2,...,55.752044,37.618500,25.079911,55.147475,342230400.0,4803915,1.612775e+09,1.614816e+09,948.45,23.622917
2,4197719,1,3,0,0,0,1,1,0,2,...,55.752044,37.618500,25.079911,55.147475,-39052800.0,4806034,1.612814e+09,1.633738e+09,2128.15,242.176389
3,4199375,1,3,0,0,0,0,1,0,2,...,55.752044,37.618500,25.079911,55.147475,-300758400.0,4807039,1.612875e+09,1.614384e+09,993.85,17.468056
4,4208819,1,3,0,0,0,3,3,2,5,...,55.752044,37.618500,25.079911,55.147475,484704000.0,4808190,1.612892e+09,1.622938e+09,1121.02,116.265278


In [57]:
df.set_index(['Client ID'], inplace=True)

In [58]:
cat_features = ['Gender', 'Year', 'Trip Type', 'Order Type', 'Country', 'Region',
       'Star Rate', 'Group Type', 'Meal Option', 'Destination City',
       'Departure City', 'Age', 'N Nights', 'N Tourists', 'Adult Tourists',
       'Baby Tourists', 'Kid Tourists', 'currency_to', 'flag_to',
       'roadinfo_to', 'timezone_name_to', 'timezone_shortname_to',
       'continent_to', 'city_to', 'country_to', 'confidence_to', 'MGRS_from',
       'currency_from', 'flag_from', 'roadinfo_from', 'timezone_name_from',
       'timezone_shortname_from', 'continent_from', 'city_from',
       'country_from', 'confidence_from', 'Order']

In [59]:
def set_cat_features(features, cat_features):
    cat_features_idx = []
    for cat_feature in cat_features:
        idx = features.index(cat_feature)
        cat_features_idx.append(idx)
    return cat_features_idx

print(set_cat_features(features=df.columns.tolist(), cat_features=cat_features))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 41]


# Цена

In [57]:
X = df.loc[:, df.columns != 'Price']
y = df['Price']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [58]:
model = catboost.CatBoostRegressor()
model.fit(X_train, y_train, verbose=500)

Learning rate set to 0.092792
0:	learn: 1620.2081366	total: 13.5ms	remaining: 13.5s
500:	learn: 685.4495442	total: 5.5s	remaining: 5.48s
999:	learn: 614.1768552	total: 10.6s	remaining: 0us


In [41]:
model.predict(X_test)

array([5337.58697916, 2760.01220912,  776.88271645, ..., 1533.07369912,
       1022.81594485, 3178.19533885])

In [61]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = model.predict(X_test)

rmse = mean_absolute_error(y_test.values, preds)
# print(np.sqrt(rmse))
rmse
# model.eval_metrics(X_test, metrics=['MAE', 'RMSE'])

416.7255787433235

In [51]:
y_test.values

array([4367.  , 2845.23,  942.77, ..., 1230.6 ,  801.  , 3001.07])

# Тип питания

In [62]:
X = df.loc[:, df.columns != 'Meal Option']
y = df['Meal Option']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [63]:
clf = catboost.CatBoostClassifier()
clf.fit(X_train, y_train, verbose=500)

Learning rate set to 0.101411
0:	learn: 1.6345281	total: 113ms	remaining: 1m 52s
500:	learn: 0.2781199	total: 41.7s	remaining: 41.6s
999:	learn: 0.2316647	total: 1m 21s	remaining: 0us


In [64]:
from sklearn.metrics import accuracy_score

preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

0.890650055507586

In [69]:
d = {}
for item in preds:
    for pred in item:
        if pred not in d:
            d[pred] = 1
        else:
            d[pred] += 1

In [71]:
X_test.shape

(32428, 23)

# Звездность

In [72]:
X = df.loc[:, df.columns != 'Star Rate']
y = df['Star Rate']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [74]:
clf_star = catboost.CatBoostClassifier()
clf._star.fit(X_train, y_train, verbose=500)

Learning rate set to 0.101411
0:	learn: 1.5252660	total: 80.5ms	remaining: 1m 20s
500:	learn: 0.2067484	total: 34.8s	remaining: 34.7s
999:	learn: 0.1698442	total: 1m 11s	remaining: 0us


In [ ]:
from sklearn.metrics import accuracy_score

preds = clf_star.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

# Продолжительность

In [86]:
X = df.loc[:, df.columns != 'N Nights']
y = df['N Nights']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

(162140, 23) (162140,)


In [89]:
clf_nights = catboost.CatBoostRegressor()
clf_nights.fit(X_train, y_train, verbose=500)

Learning rate set to 0.085345
0:	learn: 3.7583508	total: 14.3ms	remaining: 14.3s
500:	learn: 3.0459909	total: 3.86s	remaining: 3.85s
999:	learn: 2.9094843	total: 8.15s	remaining: 0us


In [88]:
from sklearn.metrics import accuracy_score

preds = clf_nights.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

0.3798815838164549

In [103]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = clf_nights.predict(X_test)
round_preds = np.around(preds)

rmse = mean_absolute_error(y_test.values, round_preds)
# rmse = mean_squared_error(y_test.values, round_preds)
# print(np.sqrt(rmse))
rmse
# model.eval_metrics(X_test, metrics=['MAE', 'RMSE'])

2.353472307882077

# Курорт

In [10]:
# -- // --

## Use class

In [75]:
from modniy_class import Reccommender

In [76]:
rec = Reccommender()

In [ ]:
report = rec.fit(train=df.dropna())